In [7]:
import os
import numpy as np
import glob
import matplotlib.pyplot as plt


from pxr import Usd
from pxr import UsdGeom
from pxr import Sdf
from pxr import Vt
from pxr import Gf
from pxr import UsdShade


In [ ]:


# generate mesh
offsets = {"hermite": 25, "linear": 0, "catmullrom": 50}
name_suffix = {"hermite": "Center", "linear": "Left", "catmullrom": "Right"}
for gender, interp in interpolated.items():
    for interp_type in interp.keys():
        xformPath = UsdGeom.Xform.Define(stage, f"/{gender}")
        mesh_interp = UsdGeom.Mesh.Define(stage, f"/{gender}/{interp_type}{name_suffix[interp_type]}")
        
        meshes = [mesh_interp]
        
        # configure
        for mesh in meshes:
            mesh.GetPointsAttr().SetVariability(Sdf.VariabilityVarying)
            mesh.GetNormalsAttr().SetVariability(Sdf.VariabilityVarying)
        
        # set mesh topology (surface)
        fvc = meshTopo[gender]["fvc"]
        fvi = meshTopo[gender]["fvi"]
        for mesh in meshes:
            mesh.GetFaceVertexCountsAttr().Set(fvc)
            mesh.GetFaceVertexIndicesAttr().Set(fvi)
        
        # start writing
        n_frames = interp[interp_type]["T"].shape[0]
        for frame in range(n_frames):
            x = interp[interp_type]["X"][frame]
            dx = np.zeros_like(x)
            dx[:, 0] += offsets[interp_type]
            x = x+dx
            if frame % 100 == 0:
                print(gender, interp_type, f"{frame}/{n_frames}", x.shape)
            mesh_interp.GetPointsAttr().Set(Vt.Vec3fArray(x.tolist()), frame)
stage.SetStartTimeCode(0)
stage.SetEndTimeCode(frame)

stage.Save()
print("Done {} frames: {}".format(n_frames, usd_path))

In [17]:
try:
    del stage
except:
    pass
out_dir = "output/01"
os.makedirs(out_dir, exist_ok=1)

usd_path = os.path.join(out_dir, "shapes.usd")
if os.path.exists(usd_path):
    os.remove(usd_path)
stage = Usd.Stage.CreateNew(usd_path)
stage_name = "Shapes"
xformPath = UsdGeom.Xform.Define(stage, f"/{stage_name}")
mesh_mean = UsdGeom.Mesh.Define(stage, f"/{stage_name}/MeanShape")
mesh_id = UsdGeom.Mesh.Define(stage, f"/{stage_name}/IdBase")
mesh_exp = UsdGeom.Mesh.Define(stage, f"/{stage_name}/ExpBase")

fvi = np.load(os.path.join(in_dir, "fvi.npy"))
n_faces = fvi.shape[0]
print("fvi:", fvi.shape)
fvi = Vt.IntArray.FromNumpy(fvi)
fvc = Vt.IntArray.FromNumpy(np.ones((n_faces))*3)
meshes = [mesh_mean, mesh_id, mesh_exp]
for mesh in meshes:
    mesh.GetPointsAttr().SetVariability(Sdf.VariabilityVarying)
    mesh.GetNormalsAttr().SetVariability(Sdf.VariabilityVarying)
    mesh.GetFaceVertexCountsAttr().Set(fvc)
    mesh.GetFaceVertexIndicesAttr().Set(fvi)


w = 2
frame = 0
in_dir = "../joon_debug"
mean_shape = np.load(os.path.join(in_dir, "mean_shape.npy"))
print("mean:{}".format(mean_shape.shape))
mesh_mean.GetPointsAttr().Set(Vt.Vec3fArray(mean_shape.tolist()), frame)

id_base = np.load(os.path.join(in_dir, "id_base.npy"))
id_base = mean_shape[:, :, None] + w*id_base
offset = 2
for i in range(id_base.shape[-1]):
    x = id_base[:, :, i].copy()
    x[:, 0] += offset
    mesh_id.GetPointsAttr().Set(Vt.Vec3fArray(x.tolist()), frame+i)
print("id_base:", id_base.shape)

exp_base = np.load(os.path.join(in_dir, "exp_base.npy"))
exp_base = mean_shape[:, :, None] + w*exp_base
print("exp_base:", exp_base.shape)

for i in range(exp_base.shape[-1]):
    x = exp_base[:, :, i].copy()
    x[:, 0] += offset*2
    mesh_exp.GetPointsAttr().Set(Vt.Vec3fArray(x.tolist()), frame+i)
stage.SetStartTimeCode(0)
stage.SetEndTimeCode(id_base.shape[-1])

stage.Save()
print("Done {} frames: {}".format(id_base.shape[-1], usd_path))



fvi: (70789, 3)
mean:(35709, 3)
id_base: (35709, 3, 80)
exp_base: (35709, 3, 64)
Done 80 frames: output/01/shapes.usd
